#### PGGM Data Science Bootcamp 2020
*Notebook by [Pedro V Hernandez Serrano](https://github.com/pedrohserrano)*

---
![](../img/image_2.png)

# 2. Intro to Internet Parsing
* [2.1. Internet parsing using API](#2.1)
* [2.2. Internet parsing using BeautifulSoup](#2.2)

---

*For the purpose of the **Use case** we will use [EDGAR Company Filings](https://www.sec.gov/edgar/searchedgar/companysearch.html)*

---
### 2.1. Internet Parsing using API
<a id="2.1">

- Before we get deep into the EDGAR data we actually need to map the CIK codes with the company names so that we can automatically can gather information
- A Central Index Key or CIK number is a number given to an individual, company, or foreign government by the United States Securities and Exchange Commission. 
- The number will help is to identify its filings in several online databases, including EDGAR.
- The numbers are ten digits in length.

**There are a couple of data sources that provide the information of companies names, ticker, CIK and Exchange schema like [rankandfiled.com](http://rankandfiled.com/#/data/tickers) or [dan.vonkohorn.com](https://dan.vonkohorn.com/2016/07/03/cik-ticker-mappings/)**

In [ ]:
import pandas as pd

In [ ]:
#reading the ticker file
cik_ticker = pd.read_csv('../datasets/cik_ticker.csv', sep='|')

In [ ]:
cik_ticker.head()

In [ ]:
cik_ticker.Exchange.unique()

In [ ]:
len(cik_ticker)

Accessing CIK - Ticker mappings via API

In [ ]:
NASDAQ = pd.read_json('https://mapping-api.herokuapp.com/exchange/NASDAQ')
OTC = pd.read_json('https://mapping-api.herokuapp.com/exchange/OTC')
NYSE = pd.read_json('https://mapping-api.herokuapp.com/exchange/NYSE')

In [ ]:
df = pd.DataFrame()
df = df.append(NASDAQ)
df = df.append(OTC)
df = df.append(NYSE)

In [ ]:
# sample for example
df = df.sample(120, random_state=1298, replace=True)

In [ ]:
df = df.head(10) #Only taking the first 10 for the moment to avoid heavy processing
len(df)

In [ ]:
df

---
#### *Learn more about API at [dataquest.io](https://www.dataquest.io/blog/python-api-tutorial/) and [freecodecamp.org](https://www.freecodecamp.org/news/what-is-an-api-in-english-please-b880a3214a82/)*

---
### 2.2. Internet Parsing using BeautifulSoup
<a id="2.2">

The main idea is to automate the search of companies and extract the information of the annual filings since there is no proper data source of annual reports we can always "scrape EDGAR" using BeautifulSoup

EDGAR data analysis sources:
- [SEC Edgar Crawler](https://github.com/coyo8/sec-edgar)
- [OpenEDGAR by LexPredict](https://github.com/LexPredict/openedgar) and [Paper](https://arxiv.org/pdf/1806.04973.pdf)     

In [ ]:
#main libraries for internet parsing
import requests
from bs4 import BeautifulSoup

In [ ]:
import re
import time
from datetime import datetime, timedelta

Let's go step by step into EDGAR database

1. In the main page we insert a ticker to look for `AAL` for example
2. Check at the URL, it has a query sintax `https://www.sec.gov/cgi-bin/browse-edgar?CIK=AAL`
3. We have the annual reports listed, ordered by the date added, the last year should be the first one

In [ ]:
# query search
cik = 'AAL'
query_search = 'https://www.sec.gov/cgi-bin/browse-edgar?CIK='+str(cik)+'&type=10-K'

In [ ]:
# internet request
page = requests.get(query_search)
page

In [ ]:
# parsing the page
page_parsed = BeautifulSoup(page.text, 'html.parser')

In [ ]:
# looking for the right doc
results_table = page_parsed.find(class_='tableFile2')

Make the procedure modular

In [ ]:
#list of cik codes
cik_codes = list(df.cik.unique())

In [ ]:
# function to get the url to the file
def get_url(query):
    page = requests.get(query)
    page_parsed = BeautifulSoup(page.text, 'html.parser')
    results_table = page_parsed.find(class_='tableFile2')
    return results_table.find_all('a')

In [ ]:
# function to get the txt files
def get_txt_files(cik_codes):
    # INPUT = list of CIK codes
    docs_urls = []
    for cik in cik_codes:
        query = 'https://www.sec.gov/cgi-bin/browse-edgar?CIK='+str(cik)+'&type=10-K'
        results_files = get_url(query)
        #if there is no files in the registry means the table and list are empty
        if results_files != []:
            #print(str(query))
            results_items = results_files[0].prettify()  #here access the company
            raw_url = 'https://www.sec.gov'+re.findall(r'\"(.+?)\"',results_items)[0]
            url = raw_url.replace('-index.htm','.txt')
            docs_urls.append(url)
        else:
            docs_urls.append('https://www.google.com/')#'no-url')
    return docs_urls

**Web crawler to extract the URL of the text files from EDGAR database**

In [ ]:
start_time = time.monotonic()
#execution
urls_files = get_txt_files(cik_codes)
#time
sttime = datetime.now().strftime('%Y%m%d_%H:%M_')
end_time = time.monotonic()
ex_time = timedelta(seconds=end_time - start_time)
print("Execution time: {}".format(ex_time))

Now we can define a new column, and add the URL of each company related to each CIK

In [ ]:
df = df.head(len(urls_files))
df['file'] = urls_files

In [ ]:
# check the first URL
urls_files[0]

---
#### *Learn more about BeautifulSoup at [dataquest.io](https://www.dataquest.io/blog/web-scraping-beautifulsoup/ ) and [BeautifulSoup official documentation](https://www.crummy.com/software/BeautifulSoup/bs4/doc/), the implementation is inspired from [scrapsfromtheloft.com](scrapsfromtheloft.com)*